# Summarise risk results for given hotspots

For each "hotspot" area:
- Value of direct cost for each single element of all infrastructure typology and for each single hazard
- Value of economic losses for each single element of all infrastructure typology and for each single hazard
- Population in numbers
- Area
- Population density (Average)
- Population and Businesses affected, disaggregated for infrastructure and hazard (if it possible for both cumulative economic losses and individual economic losses).


In [ ]:
import json
import os
from glob import glob

import geopandas
import pandas

In [ ]:
from summarise_utils import load_config

In [ ]:
base_path = load_config()['paths']['data']

## Read hotspot areas

In [ ]:
hotspots_path = os.path.join(base_path, "Incoming Data/ADN/Hotspot.shp")
hotspots_4326 = geopandas.read_file(hotspots_path) \
    .drop(columns=["Shape_Leng", "Shape_Area"]) \
    .rename(columns={"Shape_Area":"hotspot_area","Number": "hotspot_number"}) 
hotspots = hotspots_4326 \
    .to_crs(3448) # JAD2001 Jamaica metric grid EPSG:3448

## Read infrastructure and damages

In [ ]:
expected_paths = sorted(p for p in glob(os.path.join(base_path, "results/direct_damages_summary_uids/*EAD*")) if "buildings" not in p)

In [ ]:
paths = {os.path.basename(p).replace("_EAD_EAEL.parquet", ""): {"ead":p} for p in expected_paths}

In [ ]:
for p in glob(os.path.join(base_path, "processed_data/networks_uids/*/*.gpkg")):
    if "buildings" not in p:
        key = os.path.basename(p).replace(".gpkg", "")
        if f"{key}_nodes" in paths:
            paths[f"{key}_nodes"]["net"] = p
            paths[f"{key}_nodes"]["layer"] = "nodes"
        if f"{key}_edges" in paths:
            paths[f"{key}_edges"]["net"] = p
            paths[f"{key}_edges"]["layer"] = "edges"
        if f"{key}_areas" in paths:
            paths[f"{key}_areas"]["net"] = p
            paths[f"{key}_areas"]["layer"] = "areas"

In [ ]:
for key, p in paths.items():
    if key == "roads_edges":        
        network = geopandas.read_file(p["net"], layer=p["layer"]).to_crs(3448)
        network = network[["uid", "road_class", "geometry"]].rename(columns={"road_class": "asset_type"})
    elif key == "electricity_network_v3.1_nodes":
        network = geopandas.read_file(p["net"], layer=p["layer"]).to_crs(3448)
        network = network[["uid", "subtype", "geometry"]].rename(columns={"subtype": "asset_type"})
    else:
        continue
        network = geopandas.read_file(p["net"], layer=p["layer"]).to_crs(3448)
        network = network[["uid", "asset_type", "geometry"]]
    
    network_hotspot = network.sjoin(hotspots, how="inner", predicate='intersects')[["uid", "asset_type", "hotspot_number"]].set_index("uid")
    damages = pandas.read_parquet(p["ead"]).set_index("uid").join(network_hotspot, how='inner')
    hotspot_damages = damages.groupby(["hotspot_number", "asset_type", "hazard", "rcp", "epoch"]).sum()
    hotspot_damages["sector"] = key
    hotspot_damages.to_csv(f"{key}__hotspot_damages.csv")

## Read population

In [ ]:
pop = geopandas.read_file(os.path.join(base_path, "processed_data/population/population.gpkg"), layer="admin3")

In [ ]:
hotspots["area_km2"] = hotspots.geometry.apply(lambda g: g.area / 1e6)

In [ ]:
hotspot_pop = pop.overlay(hotspots, how='intersection').groupby("hotspot_number").sum()[["population"]]
hotspot_pop = hotspots.set_index("hotspot_number").join(hotspot_pop).drop(columns="geometry")

In [ ]:
hotspot_pop["population_per_km2"] = hotspot_pop.population / hotspot_pop.area_km2

In [ ]:
hotspot_pop.to_csv("population__hotspot.csv")